In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import (
    TimeoutException, 
    NoSuchWindowException, 
    NoSuchElementException, 
    StaleElementReferenceException,
    SessionNotCreatedException,
)
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
import time
from openpyxl import Workbook
from openpyxl.chart import (
    AreaChart3D,
    Reference,
    Series,
)
import os.path


def auto_login(URL, day, month, hour = "00", minute = "00", tmd = "starting at", num_month = "2"):
    
    try:
        options = Options()
#         options.add_argument('--headless')
        options.add_experimental_option('useAutomationExtension', False)
        options.add_experimental_option('excludeSwitches', ['enable-automation'])
        options.add_experimental_option('prefs', {
            'credentials_enable_service': False,
            'profile': {
            'password_manager_enabled': False
                }
        })
        
        month_mapped = {1 : 'Jan',
                        2 : 'Feb',
                        3 : 'Mar',
                        4 : 'Apr',
                        5 : 'May',
                        6 : 'Jun',
                        7 : 'Jul',
                        8 : 'Aug',
                        9 : 'Sep',
                        10 : 'Oct',
                        11 : 'Nov',
                        12 : 'Dec'}
        
        driver = webdriver.Chrome(options=options)
        driver.set_page_load_timeout(30)
        driver.get(URL)
        
        driver.execute_script("document.getElementById('ctrl-user').value='admin';")
        time.sleep(1)
        driver.execute_script("document.getElementById('ctrl-pass').value='AptG5705';")
        time.sleep(1)
        driver.execute_script("document.getElementById('ctrl-submit').click()")
         
    except TimeoutException:
        
        print(f"{URL} ,connection timeout Occured. Quiting !!!")
        driver.quit() 
        
    
    except StaleElementReferenceException:
        
        print("Please increase page loading time!")
        driver.quit()
        
    except SessionNotCreatedException:
        print("Check your network connection!!")
        driver.quit()
        
    
    finally:
        
        
        
        input_field = driver.find_element_by_id('tct')
        input_field.clear()
        input_field.send_keys(num_month)
        
        
        s1 = Select(driver.find_element_by_id('tpr'))
        s2 = Select(driver.find_element_by_id('tmd'))
        s3 = Select(driver.find_element_by_id('thr'))
        s4 = Select(driver.find_element_by_id('tmn'))
        s5 = Select(driver.find_element_by_id('tdy'))
        s6 = Select(driver.find_element_by_id('tmt'))
#         prefix = Select(driver.find_element_by_id('flpre'))
#         prefix = driver.find_element_by_xpath('//select[@id="flpre"]/option[1]')
#         prefix = driver.find_element_by_class_name("xh-highlight")
        #For month, week, day, hour, minute selection
        
        if num_month != "1":
            s1.select_by_visible_text("months")
            
        else:
            s1.select_by_visible_text("month")
#         time.sleep(3)
        ele = driver.find_element_by_xpath('//*[@id="flpre-div"]/div[2]/div/div/div[1]/div[2]/div')
        ActionChains(driver).context_click(ele).perform()
        
        time.sleep(10)
        
        s2.select_by_visible_text(tmd)
#         time.sleep(1)
        s3.select_by_visible_text(hour)
#         time.sleep(1)
        s4.select_by_visible_text(minute)
#         time.sleep(1)
        s5.select_by_value(str(day))
#         time.sleep(1)
        s6.select_by_value(str(month-1))
    
        
    
        time.sleep(3)
        driver.find_element_by_css_selector("input[type='radio'][value='2']").click()
        time.sleep(2)
        driver.find_element_by_id("flgo").click() 
        time.sleep(3)
        
        page_count = driver.find_elements_by_css_selector(".pv-page-selected")[1].text
#         print(page_count[:-5])
        
        int_page_count = int(page_count[:-5])
        
        if int(int_page_count)%50 != 0:
            total_page = int(int_page_count/50)+1
            
        else:
            total_page = int(int_page_count/50)
        
        book = Workbook()
        sheet = book.active
        sheet_title = {'A1':"Flood",
                       'B1':"Type",
                       'C1':"Sensor",
                       'D1':"Group",
                       'E1':"Detected at",
                       'F1':"Duration",
                       'G1':"Bit rate",
                       'H1':"Pkt rate",
                       'I1':"Action"}
        
        for key, value in sheet_title.items():
            sheet[key] = value
            
        all_row = []
        flood_row = []
        
        for i in range(total_page):
            time.sleep(4)
            if i == 0:
                for element in driver.find_elements_by_css_selector("td.al"):
                    if element.text != "":
                        all_row.append(element.text)
                flood_row = all_row[0::5]
                write_data(By, EC, driver, book, i*50+2, len(flood_row), i, total_page)
                get_action(By, EC, driver, book, i*50+2, len(flood_row), i, total_page)
                
            
            else:
                all_row = []
                
                next_page = driver.find_element_by_id("pages").find_element_by_link_text(str(i+1))
                next_page.click()
                time.sleep(5)
                for element in driver.find_elements_by_css_selector("td.al"):
                    if element.text != "":
                        all_row.append(element.text)
                        
                flood_row = all_row[0::5]
                time.sleep(1)
                write_data(By, EC, driver, book, i*50+2, len(flood_row), i, total_page)
                get_action(By, EC, driver, book, i*50+2, len(flood_row), i, total_page)
                
        
        sheet.title = "Event List"
        
        print("Program is closing now...")
        driver.quit()
        print("bye")
    
    
    
def write_data(by, exception, web_driver, book, start_row, row_length, current_page, total_pages):
    all_row = []
    flood_row = []
    type_row = []
    sensor_row = []
    gourp_row = []
    all_date_row = []
    detected_row =[]
    duration_row =[]
    bit_row = []
    pkt_row = []
    page_number = []
    
    
    locator = (by.CSS_SELECTOR, 'td.al')
    images =  WebDriverWait(web_driver, 30, 0.5).until(exception.presence_of_all_elements_located(locator))
    
    #web_driver.find_elements_by_css_selector("td.al")
    
    time.sleep(3)
    for element in images:
        if element.text != "":
            all_row.append(element.text)
               
    time.sleep(2)
    for element in web_driver.find_elements_by_css_selector("td.ac"):
        if element.text != "":
            all_date_row.append(element.text)
            
        
        
    sheet = book.active
        
    flood_row = all_row[0::5]
    type_row = all_row[1::5]
    sensor_row = all_row[2::5]
    group_row = all_row[3::5]
    
    detected_row = all_date_row[0::5]
    duration_row = all_date_row[1::5]
    bit_row = all_date_row[2::5]
    pkt_row = all_date_row[3::5]
            
    add_row = {'A': flood_row,
                   'B': type_row,
                   'C': sensor_row,
                   'D': group_row}
        
    x = start_row
    for i in range(row_length):
        sheet['A'+str(x)] = int(flood_row[i])
        sheet['B'+str(x)] = type_row[i]
        sheet['C'+str(x)] = sensor_row[i]
        sheet['D'+str(x)] = group_row[i]
        sheet['E'+str(x)] = detected_row[i]
        sheet['F'+str(x)] = duration_row[i]
        sheet['G'+str(x)] = bit_row[i]
        sheet['H'+str(x)] = pkt_row[i]
        x += 1
    

    book.save('/Users/lee/Desktop/SPC.xlsx')
    
    if current_page+1 != total_pages:
        print(f"\nThe page {current_page+1}/{total_pages}(total pages) data has been writed into file!")
        
    else:
        print(f"\nThe page {current_page+1}/{total_pages}(total pages) data has been writed into file!")
        print("==================================================================")
        print("All pages data were writed complete!")
        
#this function was for get and filter the action type (mitigation , blockhole)
def get_action(by, exception, web_driver, book, start_row, row_length, current_page, total_pages):
    
    all_row = []
    img_row = []

    #check all img elements were ready
    locator = (by.TAG_NAME, 'img')
    images =  WebDriverWait(web_driver, 30, 0.5).until(exception.presence_of_all_elements_located(locator))
    
    #create a sheet
    sheet = book.active
    
    #store all data except None value
    for image in images:
        if image.get_attribute('src') != None:
            img_row.append(image.get_attribute('src'))
    
    #filter the action level(mitigation or blockhole)
    x = start_row        
    for i in range(2, len(img_row)):
        if 'flag_red' in img_row[i]:
            all_row.append('BlockHole')
#             print("BlockHole!")

            
        elif 'flag_orange' in img_row[i]:
            all_row.append('Mitigation')
#             print("Migration")

    #loop all mitigation or blockhole rows and write into SPC.xlsx
    for i in range(2, len(all_row)):

        if 'BlockHole' in all_row[i]:
            sheet['I'+str(x)] = 'BlockHole'
            
        else:
            sheet['I'+str(x)] = 'Mitigation'

        x += 1
    
    #save all data after the processing
    book.save('/Users/lee/Desktop/SPC.xlsx')
        
        
if __name__ == "__main__":
    
    #auto_login(URL, day, month, hour = "00", minute = "00", tmd = "starting at", num_month = "2")
    #tmd "starting at" or "before now"
    #month index starts in 0 like Jan is 0 and so on
    auto_login("http://203.79.252.81/webui/floodactivity", 13, 10, num_month= "2")

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=86.0.4240.183)
